In [ ]:
!pip install transformers datasets rouge_score
!pip install wandb -qqq

     |████████████████████████████████| 2.3MB 2.9MB/s 
     |████████████████████████████████| 235kB 19.4MB/s 
     |████████████████████████████████| 901kB 8.4MB/s 
     |████████████████████████████████| 3.3MB 26.2MB/s 
     |████████████████████████████████| 245kB 37.2MB/s 
     |████████████████████████████████| 112kB 45.2MB/s 
     |████████████████████████████████| 1.8MB 2.9MB/s 
     |████████████████████████████████| 174kB 20.8MB/s 
     |████████████████████████████████| 102kB 8.6MB/s 
     |████████████████████████████████| 133kB 20.2MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 


In [ ]:
from collections import OrderedDict
import subprocess
import time
import xml.etree.ElementTree
import torch
import transformers
import datasets
from dataclasses import dataclass, field
from typing import Optional
import wandb
import pandas as pd

In [ ]:
# Data loading
dataset_name = 'polsum'
train_data = datasets.load_dataset(dataset_name, '3.0.0', split='train')
print(f'Length of the train set: {len(train_data)}')

Using custom data configuration 3.0.0
Reusing dataset polsum (/root/.cache/huggingface/datasets/polsum/3.0.0/1.1.0/0a1ab0e01f2e792bcf13c34a507587f31b50a759f81d161c574cfe021927b872)


Length of the train set: 569


In [ ]:
from datasets import Dataset

total_texts = len(train_data)
train_length = int(total_texts * 0.85)
val_length = int(total_texts * 0.9) - train_length
test_length = total_texts - train_length - val_length

list_of_texts_train = []
list_of_summaries_train = []
list_of_texts_val = []
list_of_summaries_val = []
list_of_texts_test = []
list_of_summaries_test = []
i = 0
for text in train_data:
    if i < train_length:
        for summary in text['summaries']['body']:
            list_of_texts_train.append(text['body'])
            list_of_summaries_train.append(summary)
    elif i < train_length + val_length:
        for summary in text['summaries']['body']:
            list_of_texts_val.append(text['body'])
            list_of_summaries_val.append(summary)
    else:
        for summary in text['summaries']['body']:
            list_of_texts_test.append(text['body'])
            list_of_summaries_test.append(summary)
    i += 1

train_dict = {}
train_dict["body_text"] = list_of_texts_train
train_dict["summary"] = list_of_summaries_train
val_dict = {}
val_dict["body_text"] = list_of_texts_val
val_dict["summary"] = list_of_summaries_val
test_dict = {}
test_dict["body_text"] = list_of_texts_test
test_dict["summary"] = list_of_summaries_test

train_data = Dataset.from_dict(train_dict)
val_data = Dataset.from_dict(val_dict)
test_data = Dataset.from_dict(test_dict)

data_dict = {'train': train_data,
 'validation': val_data,
 'test': test_data
}
print(len(train_dict["body_text"]), len(val_dict["body_text"]), len(test_dict["body_text"]))

9240 450 1155


In [ ]:
!mkdir polsum

In [ ]:
df = pd.DataFrame.from_dict(train_dict)
df.to_csv('polsum/train.csv', index=False)
df = pd.DataFrame.from_dict(val_dict)
df.to_csv('polsum/validation.csv', index=False)
df = pd.DataFrame.from_dict(test_data)
df.to_csv('polsum/test.csv', index=False)

In [ ]:
train_data = datasets.load_dataset('csv', data_files={'train': 'polsum/train.csv'})
val_data = datasets.load_dataset('csv', data_files={'validation': 'polsum/validation.csv'})
test_data = datasets.load_dataset('csv',data_files={'test': 'polsum/test.csv'})

Using custom data configuration default-a188e00adb75dbbc


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a188e00adb75dbbc/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


Using custom data configuration default-ecff5a8fb1cf0af3


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-ecff5a8fb1cf0af3/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


Using custom data configuration default-db57dcf914930e86


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-db57dcf914930e86/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [ ]:
train_data, val_data, test_data

(DatasetDict({
    train: Dataset({
        features: ['body_text', 'summary'],
        num_rows: 9240
    })
}),
 DatasetDict({
    validation: Dataset({
        features: ['body_text', 'summary'],
        num_rows: 450
    })
}),
 DatasetDict({
    test: Dataset({
        features: ['body_text', 'summary'],
        num_rows: 1155
    })
}))

In [ ]:
%env WANDB_PROJECT=psc
%env WANDB_LOG_MODEL=true
wandb.login()

env: WANDB_PROJECT=psc
env: WANDB_LOG_MODEL=true


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
with wandb.init(job_type="dataset", project='psc',    
            name="herbert-klej-polsum", entity='pns') as run:
    artifact = wandb.Artifact('polsum', type='dataset')
    artifact.add_file('polsum/train.csv', name='train.csv')
    artifact.add_file('polsum/validation.csv', name='validation.csv')
    artifact.add_file('polsum/test.csv', name='test.csv')
    wandb.log_artifact(artifact)

wandb: Currently logged in as: jkucinski (use `wandb login --relogin` to force relogin)


In [ ]:
with wandb.init(job_type="dataset", project='psc',    
            name="herbert-klej-polsum", entity='pns') as run:
    artifact = run.use_artifact('pns/psc/polsum:v0', type='dataset')
    artifact_dir = artifact.download()

wandb: Downloading large artifact polsum:v0, 129.50MB. 3 files... Done. 0:0:0


In [ ]:
train_data = datasets.load_dataset('csv', data_files={'train': artifact_dir+'/train.csv'}, split='train')
val_data = datasets.load_dataset('csv', data_files={'validation': artifact_dir+'/validation.csv'}, split='validation')
test_data = datasets.load_dataset('csv',data_files={'test': artifact_dir+'/test.csv'}, split='test')

Using custom data configuration default-bad27331042ca3c5
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-bad27331042ca3c5/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)
Using custom data configuration default-7fc8fa83072b9358
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-7fc8fa83072b9358/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)
Using custom data configuration default-09144b972fea9cd2
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-09144b972fea9cd2/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)


In [ ]:
train_data, val_data, test_data

(Dataset({
     features: ['body_text', 'summary'],
     num_rows: 9240
 }), Dataset({
     features: ['body_text', 'summary'],
     num_rows: 450
 }), Dataset({
     features: ['body_text', 'summary'],
     num_rows: 1155
 }))